In [5]:
import os
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time

# Function to extract href attribute values from anchor tags, skipping header rows
def url_extract(tds):
    results = []
    for td in tds:
        # Skip rows with specific attributes or text content indicating header rows
        if td.has_attr('aria-label') or 'header' in td.get('class', []) or 'header' in td.get('class', []):
            continue
        a_tag = td.find('a')  # Find the <a> tag within the <td> or <th>
        if a_tag:
            href_value = a_tag['href']  # Get the value of the href attribute
            results.append(href_value)
        else:
            results.append(None)
    return results

# Define headers for different types of data
combine_header = ['player', 'pos', 'college', 'stats', 'height', 'weight', 'forty', 'vertical', 'bench', 'broad', 'threecone', 'shuttle', 'drafted']

# Create directory for caching HTML files
cache_dir = 'cache'
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

# Function to read HTML content with caching
def read_html_cache(url, year):
    fn = f"{year}_combine.htm"
    fn_path = os.path.join(cache_dir, fn)
    if not os.path.exists(fn_path):
        with urlopen(url) as response:
            html = response.read().decode('utf-8')
            with open(fn_path, 'w') as f:
                f.write(html)
    with open(fn_path, 'r') as f:
        return BeautifulSoup(f.read(), 'html.parser')

# Function to extract player URLs from table cells
def player_url_extract(tds):
    results = []
    for td in tds:
        # Find 'a' tag within 'td' and extract the href attribute
        if td.find('a'):
            player_url = td.find('a')['href']
            if player_url.startswith('/players'):
                # Construct full player URL and append to results
                full_url = f"http://www.pro-football-reference.com{player_url}"
                results.append(full_url)
            else:
                results.append(None)
        else:
            results.append(None)
    return results

# Scrape combine data with player names
combine_data = []
for year in range(2000, 2024):  # Adjust the range as needed
    time.sleep(4)
    url = f'http://www.pro-football-reference.com/draft/{year}-combine.htm'
    soup = read_html_cache(url, year)
    tables = soup.find_all('table')
    college_cells = tables[0].find_all('td', {'data-stat': 'college'})  # Adjusted to target 'college' attribute
    player_cells = tables[0].find_all(['td', 'th'], {'data-stat': 'player'})  # Target both 'td' and 'th' elements
    college_urls = url_extract(college_cells)
    player_names = url_extract(player_cells)  # Extract player names using the modified function
    combine_table = pd.read_html(str(tables[0]))[0]
    combine_table.columns = combine_header
    combine_table = combine_table[combine_table['pos'] != 'Pos']

    # Add new columns for college stats and player names
    combine_table['college_stats_url'] = college_urls
    combine_table['nfl_stats'] = player_names

    # Add a new column 'Year' to the DataFrame
    combine_table['Year'] = year

    combine_data.append(combine_table)

# Concatenate all combine data DataFrames
combine_table = pd.concat(combine_data, ignore_index=True)






In [6]:
import pandas as pd


# Apply conditional check and concatenate URL prefix
combine_table['nfl_stats'] = combine_table['nfl_stats'].apply(lambda x: f"https://www.pro-football-reference.com/{x}" if x else None)


In [7]:
combine_table


,player,pos,college,stats,height,weight,forty,vertical,bench,broad,threecone,shuttle,drafted,college_stats_url,nfl_stats,Year
0,John Abraham,OLB,South Carolina,NaN,6-4,252,4.55,NaN,NaN,NaN,NaN,NaN,New York Jets / 1st / 13th pick / 2000,None,https://www.pro-football-reference.com//player...,2000
1,Shaun Alexander,RB,Alabama,College Stats,6-0,218,4.58,NaN,NaN,NaN,NaN,NaN,Seattle Seahawks / 1st / 19th pick / 2000,https://www.sports-reference.com/cfb/players/s...,https://www.pro-football-reference.com//player...,2000
2,Darnell Alford,OT,Boston Col.,NaN,6-4,334,5.56,25.0,23,94,8.48,4.98,Kansas City Chiefs / 6th / 188th pick / 2000,None,https://www.pro-football-reference.com//player...,2000
3,Kyle Allamon,TE,Texas Tech,NaN,6-2,253,4.97,29.0,NaN,104,7.29,4.49,NaN,None,None,2000
4,Rashard Anderson,CB,Jackson State,NaN,6-2,206,4.55,34.0,NaN,123,7.18,4.15,Carolina Panthers / 1st / 23rd pick / 2000,None,https://www.pro-football-reference.com//player...,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7994,Luke Wypler,C,Ohio St.,College Stats,6-3,303,5.14,30.5,NaN,106,7.64,4.53,Cleveland Browns / 6th / 190th pick / 2023,https://www.sports-reference.com/cfb/players/l...,https://www.pro-football-reference.com//player...,2023
7995,Bryce Young,QB,Alabama,College Stats,5-10,204,NaN,NaN,NaN,NaN,NaN,NaN,Carolina Panthers / 1st / 1st pick / 2023,https://www.sports-reference.com/cfb/players/b...,https://www.pro-football-reference.com//player...,2023
7996,Byron Young,DT,Alabama,College Stats,6-3,294,NaN,26.0,24,108,7.68,NaN,Las Vegas Raiders / 3rd / 70th pick / 2023,https://www.sports-reference.com/cfb/players/b...,https://www.pro-football-reference.com//player...,2023
7997,Byron Young,EDGE,Tennessee,College Stats,6-2,250,4.43,38.0,22,132,7.19,NaN,Los Angeles Rams / 3rd / 77th pick / 2023,https://www.sports-reference.com/cfb/players/b...,https://www.pro-football-reference.com//player...,2023


In [8]:
# Save draft_table to a CSV file
combine_table.to_csv('combine_data.csv', index=False)